# Export raw tick data using dedicated service - Python

### Overview
This sample demonstrates how to request from a dedicated raw data service **on-demand** tick by tick data `for a given instrument or a given watchlist`. 

### Inputs/outputs
Tick data extraction sample requires instrument's identifier, date time intervals as per inputs and exports tick by tick data file. 

### Services used
This sample uses *gRPC requests* in order to retrieve tick by tick dataset from the hosted service. The queried endpoint in this script are:
* *TickRawService*: to directly retrieve raw dataset from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    
### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    
***

# Run tick by tick dataset extraction sample

### Step 1: Install packages

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.level_pb2 as level
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_raw_pb2 as tick_raw
import systemathics.apis.services.tick.v1.tick_raw_pb2_grpc as tick_raw_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Retrieve data
To request *tick raw* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick raw request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 Ticks level

In [ ]:
# set the ticks level: trades or trades and book quotes
level = level.LEVEL_TRADES_AND_BOOK 

#### 3.3 Time period delimitation

#### Open Hours for Euronext

[Euronext Trading Hours](https://www.euronext.com/en/trade/trading-hours-holidays)

|Open Hours| Trading | Close | Trading At Last |
| -------- | --------| ------|-----------------|
| 09:00 (CET) | 09:00 - 17:30 (CET) | 17:35 (CET) | 17:40 (CET)|
| 07:00 (UTC) | 07:00 - 15:30 (UTC) | 15:35 (UTC) | 15:40 (UTC)|

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 9, day = 13), 
    end_date = date.Date(year = 2021, month = 9, day = 13)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval1 = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 6, minutes = 50, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 7, minutes = 15, seconds = 0)
)
time_interval2 = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 15, minutes = 20, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 15, minutes = 45, seconds = 0)
)

In [ ]:
date_interval.start_date

#### 3.4 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

The following code snippets create the market data request and instantiate the service:

In [ ]:
# generate tick raw request
request = tick_raw.TickRawRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = constraints,
    level = level
)

#### 3.5 Export raw tick data
Tick by tick data is sent through a *stream*, the following code snippet exports tick by tick dataset in a file in the same folder as this sample:

In [ ]:
import csv
from datetime import timedelta

filename = 'raw_ticks_{0}-{1}.csv'.format(ticker, exchange)
ticks_count = 0

with open(filename, mode='w') as raw_ticks_file:
    raw_ticks_writer = csv.writer(raw_ticks_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # write 1rst row
    raw_ticks_writer.writerow(['Timestamp', 'Tick'])
    
    # open a gRPC channel
    with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
        credentials = grpc.ssl_channel_credentials(f.read())

    with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:

        # instantiate the tick raw service
        service = tick_raw_service.TickRawServiceStub(channel)

        # process the tick raw request
        for items in service.TickRaw(request=request, metadata=[('authorization', token)]):
            for item in items.raw_items:
                time = datetime.fromtimestamp(item.time_stamp.seconds) + timedelta(microseconds = item.time_stamp.nanos/1000)
                tick = str(item.payload, 'UTF-8')[2:]
                raw_ticks_writer.writerow([time,tick])
                ticks_count = ticks_count + 1
            